# **Deep Learning**
## **Goodfellow, Bengio, and Courville**

Christopher La Valle

## **1 - Introduction**

## **2 - Linear Algebra**

## **3 - Probability and Information Theory**

## **4 - Numerical Computation**

See numerical linear algebra notebook

Machine learning algorithms usually require a high amount of numerical computation. Thys typcially refers to algorithms that sovle mathematical problems by methods that update estimates of the solution via an iterative process, rather than analytically deriving a formula to provide a symbolic expression for the correct solution. Even just evaluating a mathematical function on a digital computer can be difficult when the function involves real numbers, which cannot be represented precisely using a finite amount of memory.

### **4.1 - Overflow and Underflow**


The fundamental difficulty in performing continous math on a digital computer is that we need to represent infinitely many real numbers with a finite number of bit patterns. This means that for almsost al lreal numbers, we incur some approximation error when we represent the number in the computer. In many cases, this is just round error. Rounding error is problematic, especially when it compounds across many operations, and can cause algorithms that work in theory  to fail in practice if they are not designed to minimize the accumulation of rounding error.

**Underflow** occurs when numbers near zero are rounded to zero. Many functions behave qualitatively differently when their argument is zero rather than small positive number. 

**Overflow** occurs when numbers with large magnitude are approximated as $\infty$ or $-\infty$. Further arithmetic will usually change these infinite values into not-a-number values.

One example of a function that must be stabilized against underflow and overflow is the **softmax function**. The softmax function is often used to predict the probabilities assoicated with a multinoulli distribution. The softmax function is defined to be

$$\text{softmax}(x_i)=\frac{\exp(x_i)}{\sum_{j=1}^n\exp(x_j)}\tag{4.1}$$

Consider what happens when al lthe $x_i$ are equal to some constant $c$. Analytically, we can see that all the outputs should be equal to $\frac{1}{n}$. Numerically, this may not occur when $c$ has large magnitude. If $c$ is very negative, then $\exp(c)$ will underflow. This means the denominatior of the softmax will become $0$, so the final result is undefined. When $c$ is very large and positive, $\exp(c)$ will overflow, again resulting the expression as a whole being undefined. Both of these difficulties can be resolved by instead evaluating $\text{softmax}(z)$ where $z=x-\max_ix_i$. Simple algebra shows that the value of the softmax function is not changed analytically by adding or subtracting a scalar from the input vector. Subtracting $\max_ix_i$ results in the largest argument to $\exp$ being $0$, which rules out the possibility of overflow. LIkewise, at least one term in the denominiaotr has a value of $1$, which rules out the posssibility of under flow in the denominatior leading to a division by zero.

There is still one small problem. Underflow in the numerator can still cause the expression as a whole to evaluate to zero. This means that if we impliment $\log\text{softmax}(x)$ by first running the fotmax subroutine then passing the result to the log function, we could erroneously obtain $-\infty$. Instead, we must implement a separate function that calculates $\log\text{softmax}$ in a numerically stable way. The $\log\text{softmax}$ function can be stabilized using the same trick as we used to stabilize the softmax function.

Theano (Bergstra et al., 2010 Bastien et al. 2012) is an example of a software package that automatically detects and stabilizes many common numerically unstable expressions that arise in the context of deep learning.

### **4.2 - Poor Conditioning**

Conditioning refers to how rapidly a function changes with respect to small changes in its inputs. Functions that change rapidly when their inputs are perturbed slightly can be problematic for scientific computaiton because rounding errors in the inputs can result in large changes in the output.

Consider the function $f(x)=\mathbf{A}^{-1}\mathbf{x}$. When $\mathbf{A}\in\mathbb{R}^{n\times n}$ has an eigenvalue decomposition, the **condition number* is

$$\underset{i,j}{\max}\left|\frac{\lambda_i}{\lambda_j}\right|\tag{4.2}$$

This is the ratio of the magnitude of the largest and smallest eigenvalue. When this number is large, matrix inversion is particularly sensitive to error in the input.

This sensitivity is an intrinsic property of the matrix itself, not the result of rounding error during matrix inversion. Poorly conditioned matrices amplify pre-exisitng errors when we multiply by the true matrix inverse. In practice, the error will be compounded furhter by numerical errors in the inversion process itself.

### **4.3 - Gradient-Based Optimization**

Most dep learning algorithms involve optimizations of osme sort. Optimization refers to the task of either minimizing or maximizing some function $f(\mathbf{x})$ by altering $\mathbf{x}$. We usually phrase most optimization problems in terms of minimizing $f(\mathbf{x})$. Maximization may be axxomplished via a minimization algorithm by minimizing $-f(\mathbf{x})$.

The function we want to minize of maximize is called the **objective function**, or **criterion**. When we are minimizing it, we may also call it the **cost function**, **loss function**, or **error function**. 

We often denote the value that minimizes or maximizes a function with a superscript $*$: $\mathbf{x}^*=\arg\min f(\mathbf{x})$ for example.

Suppose we have a function $y=f(x)$, where both $x$ and $Y$ are real numbers. The **derivative*